# Labelme转YOLO-Keypoint-单个文件

同济子豪兄 2023-4-16

## 导入工具包

In [1]:
import os
import json
import numpy as np

## 数据集类别信息

In [2]:
# 框的类别
bbox_class = {
    'sjb_rect':0  
}

In [3]:
# 关键点的类别
keypoint_class = ['angle_30', 'angle_60', 'angle_90']

## 载入一个labelme格式的json标注文件

In [4]:
labelme_path = 'DSC_0281.json'

In [6]:
with open(labelme_path, 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [7]:
labelme.keys()

dict_keys(['version', 'flags', 'shapes', 'imagePath', 'imageData', 'imageHeight', 'imageWidth'])

### 元数据 

In [8]:
labelme['version']

'5.1.1'

In [9]:
labelme['flags']

{}

In [10]:
# 图像文件名
labelme['imagePath']

'DSC_0281.jpg'

In [11]:
labelme['imageData']

In [12]:
# 图像高度
labelme['imageHeight']

3712

In [13]:
# 图像宽度
labelme['imageWidth']

5568

### 标注信息

In [14]:
labelme['shapes']

[{'label': 'sjb_rect',
  'points': [[1138.0, 0.0], [4480.0, 1562.0]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'sjb_rect',
  'points': [[1160.0, 1954.0], [4560.0, 3566.0]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'angle_30',
  'points': [[4434.0, 284.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_30',
  'points': [[4526.0, 2316.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_60',
  'points': [[1202.0, 44.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_60',
  'points': [[1218.0, 2000.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_90',
  'points': [[1934.0, 1512.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_90',
  'points': [[1906.0, 3532.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'sjb_poly',
  'points':

## 生成YOLO格式的标注文件

In [15]:
img_width = labelme['imageWidth']   # 图像宽度
img_height = labelme['imageHeight'] # 图像高度

# 生成 YOLO 格式的 txt 文件
suffix = labelme_path.split('.')[-2]
yolo_txt_path = suffix + '.txt'

## 遍历每个标注，如果遇到框，就找到该框里所有的关键点，并按顺序写入txt文件

In [16]:
with open(yolo_txt_path, 'w', encoding='utf-8') as f:
    
    for each_ann in labelme['shapes']: # 遍历每个标注

        if each_ann['shape_type'] == 'rectangle': # 如果遇到框
            
            yolo_str = ''
            
            ## 框的信息
            # 框的类别 ID
            bbox_class_id = bbox_class[each_ann['label']]
            yolo_str += '{} '.format(bbox_class_id)
            # 左上角和右下角的 XY 像素坐标
            bbox_top_left_x = int(min(each_ann['points'][0][0], each_ann['points'][1][0]))
            bbox_bottom_right_x = int(max(each_ann['points'][0][0], each_ann['points'][1][0]))
            bbox_top_left_y = int(min(each_ann['points'][0][1], each_ann['points'][1][1]))
            bbox_bottom_right_y = int(max(each_ann['points'][0][1], each_ann['points'][1][1]))
            # 框中心点的 XY 像素坐标
            bbox_center_x = int((bbox_top_left_x + bbox_bottom_right_x) / 2)
            bbox_center_y = int((bbox_top_left_y + bbox_bottom_right_y) / 2)
            # 框宽度
            bbox_width = bbox_bottom_right_x - bbox_top_left_x
            # 框高度
            bbox_height = bbox_bottom_right_y - bbox_top_left_y
            # 框中心点归一化坐标
            bbox_center_x_norm = bbox_center_x / img_width
            bbox_center_y_norm = bbox_center_y / img_height
            # 框归一化宽度
            bbox_width_norm = bbox_width / img_width
            # 框归一化高度
            bbox_height_norm = bbox_height / img_height
            
            yolo_str += '{:.5f} {:.5f} {:.5f} {:.5f} '.format(bbox_center_x_norm, bbox_center_y_norm, bbox_width_norm, bbox_height_norm)
            
            ## 找到该框中所有关键点，存在字典 bbox_keypoints_dict 中
            bbox_keypoints_dict = {}
            for each_ann in labelme['shapes']: # 遍历所有标注
                if each_ann['shape_type'] == 'point': # 筛选出关键点标注
                    # 关键点XY坐标、类别
                    x = int(each_ann['points'][0][0])
                    y = int(each_ann['points'][0][1])
                    label = each_ann['label']
                    if (x>bbox_top_left_x) & (x<bbox_bottom_right_x) & (y<bbox_bottom_right_y) & (y>bbox_top_left_y): # 筛选出在该个体框中的关键点
                        bbox_keypoints_dict[label] = [x, y]
            
            ## 把关键点按顺序排好
            for each_class in keypoint_class: # 遍历每一类关键点
                if each_class in bbox_keypoints_dict:
                    keypoint_x_norm = bbox_keypoints_dict[each_class][0] / img_width
                    keypoint_y_norm = bbox_keypoints_dict[each_class][1] / img_height
                    yolo_str += '{:.5f} {:.5f} {} '.format(keypoint_x_norm, keypoint_y_norm, 2) # 2-可见不遮挡 1-遮挡 0-没有点
                else: # 不存在的点，一律为0
                    yolo_str += '0 0 0 '.format(keypoint_x_norm, keypoint_y_norm, 0)
            # 写入 txt 文件中
            f.write(yolo_str + '\n')
            
print('{} --> {} 转换完成'.format(labelme_path, yolo_txt_path))

DSC_0281.json --> DSC_0281.txt 转换完成
